# 제로베이스데이터사이언스 스쿨 원데이 프로젝트
> ## 세계 테러 데이터 분석 - SQL 사용하기

---

#### 총 8문제 100점 만점 

- 문제 1 10점  
- 문제 2,3,5,6,7,8번 각 10점 
- 문제 4번 30점
- 총점 100점 

#### 유의사항

- 모든 정답 코드는 jupyter notebook 파일에 작성해주세요. 
- 1번 문제는 정답 코드를 그대로 공개해드립니다. 따라서 진행해주세요. 
- 그 외 문제들은 참고하실 수 있도록 주석과 출력 코드를 보여드립니다. 
- 제출시, 반드시 결과가 실행되어 있어야 합니다. 미실행 또는 오류시 감점입니다. 
- 제출시, [DS]sql3_choyongha.ipynb 형식에 맞춰 제출해주세요.


---

여러분은 이전 과제에서 세계 테러 데이터 분석 프로젝트를 진행하였습니다. 

이번 시간에는 분석할 데이터를 Database 로 변환하고, 분석에 필요한 데이터를 저장된 Database 에서 쿼리하여 가져와 보도록 하겠습니다. 

#### 작업환경

- aws rds 사용시: 이전 과제에서 생성한 AWS RDS (MySQL) 의 Database 및 사용자를 사용하세요. 
  - Database Name: oneday 
  - User Name / Password: oneday / 1234
- local 사용시: 로컬 환경에서 생성한 Database 및 사용자를 사용하세요.
  - 위와 동일하게 생성해주세요

## 원본 Data Download
- https://www.kaggle.com/datasets/START-UMD/gtd

---

문제 1.  csv 파일에 저장된 세계 테러 데이터를 하나의 테이블에 저장하세요. 

- globalterrorismmdb_0718.csv - https://www.kaggle.com/datasets/START-UMD/gtd
- pandas의 to_sql 함수 사용 (힌트 : create_engine)
- Table name : origin_terror_data 

제출 1. Table 생성 및 조회 결과 : DESC origin_terror_data, SELECT  COUNT(*) FROM origin_terror_data

제출 2. 관련 코드 및 실행 결과 (ipynb)



In [1]:
import mysql.connector

In [171]:
##############################################################################################
#  문제 1. 원본 데이터 (csv)를 데이터베이스로 변환하고 확인하기 
#  globalterrorismdb_0718dist.csv -> origin_terror_data (table)
#  Pandas 의 to_sql 사용 
##############################################################################################

##############################################################################################
#  pip install pymysql
#  pip install SQLAlchemy Flask-SQLAlchemy
##############################################################################################

import time
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import configparser

# 데이터 불러오기
df = pd.read_csv('./globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')

# DB 접속 엔진 객체 생성
user = 'oneday'
password = '1234'
host = 'database-1.cn4vthwm7kuj.us-east-2.rds.amazonaws.com'
port = 3306
database = 'oneday'

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4')

# DB 테이블 명(생성될 테이블 이름)
table_name = "origin_terror_data"

# DB에 DataFrame 적재
df.to_sql(index = False, 
          name = table_name,
          con = engine,
          if_exists = 'append',
          method = 'multi', 
          chunksize = 10000)


In [172]:
records = engine.execute("SELECT COUNT(*) FROM origin_terror_data").fetchall()
print(records)

[(181691,)]


In [3]:
query = 'DESC origin_terror_data;'
cursor.execute(query)

records = cursor.fetchall()
print(records)

[('eventid', b'bigint', 'YES', '', None, ''), ('iyear', b'bigint', 'YES', '', None, ''), ('imonth', b'bigint', 'YES', '', None, ''), ('iday', b'bigint', 'YES', '', None, ''), ('approxdate', b'text', 'YES', '', None, ''), ('extended', b'bigint', 'YES', '', None, ''), ('resolution', b'text', 'YES', '', None, ''), ('country', b'bigint', 'YES', '', None, ''), ('country_txt', b'text', 'YES', '', None, ''), ('region', b'bigint', 'YES', '', None, ''), ('region_txt', b'text', 'YES', '', None, ''), ('provstate', b'text', 'YES', '', None, ''), ('city', b'text', 'YES', '', None, ''), ('latitude', b'double', 'YES', '', None, ''), ('longitude', b'double', 'YES', '', None, ''), ('specificity', b'double', 'YES', '', None, ''), ('vicinity', b'bigint', 'YES', '', None, ''), ('location', b'text', 'YES', '', None, ''), ('summary', b'text', 'YES', '', None, ''), ('crit1', b'bigint', 'YES', '', None, ''), ('crit2', b'bigint', 'YES', '', None, ''), ('crit3', b'bigint', 'YES', '', None, ''), ('doubtterr', b'

In [4]:
conn.close()

문제 2. origin_terror_data 에서 region, country 관련 데이터는 code 와 txt (name) 속성으로 정의되어 있습니다. 

문제 2-1. Region 및 Country 테이블을 그림과 같은 구조로 생성하세요. 

- origin_terror_data 를 분석하여 각 테이블의 데이터 타입을 정의하세요. 
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요. 
- Region 과 Country 데이터 사이의 관계를 파악하여 Foreign Key 를 설정하세요.

참고> 
- Region.region_code     = origin_terror_data.region
- Region.region_name     = origin_terror_data.region_txt
- Country.country_code   = origin_terror_data.country
- Country.country_name   = origin_terror_data.country_txt


In [155]:
##############################################################################################
#  문제 2. region / country / city 데이터 추출하여 데이터베이스로 변환하기 
#  중복을 제거한 code - name 값 
#  region - country - city 관계 정의 
##############################################################################################
conn = mysql.connector.connect(
    host = 'database-1.cn4vthwm7kuj.us-east-2.rds.amazonaws.com',
    user = 'oneday',
    password = '1234',
    database = 'oneday'
)

cursor = conn.cursor(buffered=True)

In [32]:
# Region 테이블 생성 
region_table_query = """
CREATE TABLE Region (
    region_code int PRIMARY KEY,
    region_name VARCHAR(32) NOT NULL
)
"""
cursor.execute(region_table_query)

# Country 테이블 생성
country_table_query = """
CREATE TABLE Country (
    country_code int PRIMARY KEY,
    region_code int NOT NULL,
    country_name VARCHAR(32) NOT NULL,
    FOREIGN KEY (region_code) REFERENCES Region(region_code)
)

"""
cursor.execute(country_table_query)


In [33]:
# region_txt 의 max length 체크 
region_txt_size_query = "SELECT MAX(LENGTH(region_txt)) FROM origin_terror_data"
cursor.execute(region_txt_size_query)
region_txt_size = cursor.fetchone()
print([region_txt_size])

[(27,)]


In [46]:
# Region 테이블 만들기 
query = 'DESC Region;'
cursor.execute(query)

region_table = cursor.fetchall()
print(region_table)

[('region_code', b'int', 'NO', 'PRI', None, ''), ('region_name', b'varchar(32)', 'NO', '', None, '')]


In [35]:
# country_txt 의 max length 체크 
country_txt_query = "SELECT MAX(LENGTH(country_txt)) FROM origin_terror_data"
cursor.execute(country_txt_query)
country_txt_size = cursor.fetchone()
print([country_txt_size])

[(32,)]


In [41]:
# Country 테이블 만들기  
query = 'DESC Country;'
cursor.execute(query)

country_table = cursor.fetchall()
print(country_table)

[('country_code', b'int', 'NO', 'PRI', None, ''), ('region_code', b'int', 'NO', 'MUL', None, ''), ('country_name', b'varchar(32)', 'NO', '', None, '')]


문제 2-2. origin_terror_data 테이블에서 Region 및 Country 데이터를 추출하여 문제 2-1.에서 생성한 테이블에 입력하고 확인하세요.   

- 중복을 제거한 Unique Data 를 추출하세요. 
- 데이터를 INSERT 할때 순서를 고민하세요. 

제출 3. Table 생성 및 조회 결과 : DESC Region, DESC Country

제출 4. 관련 코드 및 실행 결과 (ipynb)

In [182]:
# region 테이블에 이볅
region_query = """
    INSERT INTO Region (region_code, region_name)
    SELECT DISTINCT region, region_txt
    FROM origin_terror_data
    WHERE region IS NOT NULL AND region_txt IS NOT NULL
    ON DUPLICATE KEY UPDATE region_name=VALUES(region_name)
"""
cursor.execute(region_query)
conn.commit()

In [183]:
# Country 테이블에 입력
country_query = """
    INSERT INTO Country (country_code, country_name, region_code)
    SELECT od.country, od.country_txt, r.region_code
    FROM origin_terror_data od
    JOIN Region r ON od.region_txt = r.region_name
    GROUP BY od.country, od.country_txt, r.region_code
"""

cursor.execute(country_query)
conn.commit()

In [70]:
# Region Unique Data 조회
desc_region_query = """
    SELECT DISTINCT region_code, region_name
    FROM Region
"""
cursor.execute(desc_region_query)
print(cursor.fetchall())

[(1, 'North America'), (2, 'Central America & Caribbean'), (3, 'South America'), (4, 'East Asia'), (5, 'Southeast Asia'), (6, 'South Asia'), (7, 'Central Asia'), (8, 'Western Europe'), (9, 'Eastern Europe'), (10, 'Middle East & North Africa'), (11, 'Sub-Saharan Africa'), (12, 'Australasia & Oceania')]


In [77]:
# Country Unique Data 조회
desc_country_query = """
    SELECT DISTINCT country_code, country_name
    FROM Country
"""
cursor.execute(desc_country_query)
print(cursor.fetchall())

[(4, 'Afghanistan'), (5, 'Albania'), (6, 'Algeria'), (7, 'Andorra'), (8, 'Angola'), (10, 'Antigua and Barbuda'), (11, 'Argentina'), (12, 'Armenia'), (14, 'Australia'), (15, 'Austria'), (16, 'Azerbaijan'), (17, 'Bahamas'), (18, 'Bahrain'), (19, 'Bangladesh'), (20, 'Barbados'), (21, 'Belgium'), (22, 'Belize'), (23, 'Benin'), (25, 'Bhutan'), (26, 'Bolivia'), (28, 'Bosnia-Herzegovina'), (29, 'Botswana'), (30, 'Brazil'), (31, 'Brunei'), (32, 'Bulgaria'), (33, 'Burkina Faso'), (34, 'Burundi'), (35, 'Belarus'), (36, 'Cambodia'), (37, 'Cameroon'), (38, 'Canada'), (41, 'Central African Republic'), (42, 'Chad'), (43, 'Chile'), (44, 'China'), (45, 'Colombia'), (46, 'Comoros'), (47, 'Republic of the Congo'), (49, 'Costa Rica'), (50, 'Croatia'), (51, 'Cuba'), (53, 'Cyprus'), (54, 'Czech Republic'), (55, 'Denmark'), (56, 'Djibouti'), (57, 'Dominica'), (58, 'Dominican Republic'), (59, 'Ecuador'), (60, 'Egypt'), (61, 'El Salvador'), (62, 'Equatorial Guinea'), (63, 'Eritrea'), (64, 'Estonia'), (65, 'Et

In [78]:
# Region 테이블 조회
desc_region_query = """
    DESC Region
"""
cursor.execute(desc_region_query)
print(cursor.fetchall())

[('region_code', b'int', 'NO', 'PRI', None, ''), ('region_name', b'varchar(32)', 'NO', '', None, '')]


In [79]:
# Country 테이블 조회
desc_country_query = """
    DESC Country
"""
cursor.execute(desc_country_query)
print(cursor.fetchall())

[('country_code', b'int', 'NO', 'PRI', None, ''), ('region_code', b'int', 'NO', 'MUL', None, ''), ('country_name', b'varchar(32)', 'NO', '', None, '')]


문제 3.  origin_terror_data 에서 attack type, target type, weapon type 관련 데이터는 code 와 txt 속성으로 정의되어 있습니다.  

문제 3-1. AttackType, TargetType, WeaponType 테이블을 그림과 같은 구조로 생성하세요. 

- origin_terror_data 를 분석하여 각 테이블의 데이터 타입을 정의하세요. 
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요. 

참고>
- AttackType.attacktype_code    = origin_terror_data.attacktype1
- AttackType.attacktype_desc    = origin_terror_data.attacktype1_txt 
- TargetType.targtype_code      = origin_terror_data.targtype1
- TargetType.targtype_desc      = origin_terror_data.targtype1_txt
- WeaponType.weaptype_code      = origin_terror_data.weaptype1
- WeaponType.weaptype_desc      = origin_terror_data.weaptype1_txt 

In [185]:
# AttackType 테이블
cursor.execute("""
    CREATE TABLE AttackType (
        attacktype_code INT PRIMARY KEY,
        attacktype_desc VARCHAR(35) NOT NULL
    )
""")
conn.commit()

# TargetType 테이블
cursor.execute("""
    CREATE TABLE TargetType (
        targtype_code INT PRIMARY KEY,
        targtype_desc VARCHAR(32) NOT NULL
    )
""")
conn.commit()

# WeaponType 테이블
cursor.execute("""
    CREATE TABLE WeaponType (
        weaptype_code INT PRIMARY KEY,
        weaptype_desc VARCHAR(80) NOT NULL
    )
""")
conn.commit()

ProgrammingError: 1050 (42S01): Table 'AttackType' already exists

In [186]:
cursor.execute("DESCRIBE AttackType")
print(cursor.fetchall())

cursor.execute("DESCRIBE TargetType")
print(cursor.fetchall())

cursor.execute("DESCRIBE WeaponType")
print(cursor.fetchall())

[('attacktype_code', b'int', 'NO', 'PRI', None, ''), ('attacktype_desc', b'varchar(35)', 'NO', '', None, '')]
[('targtype_code', b'int', 'NO', 'PRI', None, ''), ('targtype_desc', b'varchar(32)', 'NO', '', None, '')]
[('weaptype_code', b'int', 'NO', 'PRI', None, ''), ('weaptype_desc', b'varchar(80)', 'NO', '', None, '')]


In [88]:
# attacktype1 의 max length 체크 
attacktype1_max_length = 'SELECT MAX(LENGTH(attacktype1_txt)) FROM origin_terror_data;'
cursor.execute(attacktype1_max_length)
print(cursor.fetchall())

[(35,)]


In [95]:
# targettype1 의 max length 체크 
targettype1_max_length = ' SELECT MAX(LENGTH(targtype1_txt)) FROM origin_terror_data'
cursor.execute(targettype1_max_length)
print(cursor.fetchall())

[(30,)]


In [96]:
# weaptype1 의 max length 체크 
weaptype1_txt_query = """
    SELECT MAX(LENGTH(weaptype1_txt)) FROM origin_terror_data
"""
cursor.execute(weaptype1_txt_query)
print(cursor.fetchall())

[(75,)]


문제 3-2. origin_terror_data 테이블에서 Attack Type, Target Type, Weapon Type 데이터를 추출하여 문제 3-1.에서 생성한 테이블에 입력하고 확인하세요.   

- 중복을 제거한 Unique Data 를 추출하세요. 
- 데이터를 INSERT 할때 순서를 고민하세요.

제출 5. Table 생성 및 조회 결과 : DESC AttackType, DESC TargetType, WeaponType

제출 6. 관련 코드 및 실행 결과 (ipynb) 

In [190]:
# AttackType 테이블에 데이터 입력
insert_query = """
    INSERT INTO AttackType (attacktype_code, attacktype_desc) VALUES (%s, %s)
"""

for data in attack_type_data:
    cursor.execute(insert_query, data)
    conn.commit()

In [105]:
# 중복을 제거한 Unique Attack Type 데이터 추출
attack_type_query = """
    SELECT DISTINCT attacktype1, attacktype1_txt FROM origin_terror_data 
    WHERE attacktype1 IS NOT NULL ORDER BY attacktype1
"""
cursor.execute(attack_type_query)
attack_type_data = cursor.fetchall()
print(attack_type_data)

[(1, 'Assassination'), (2, 'Armed Assault'), (3, 'Bombing/Explosion'), (4, 'Hijacking'), (5, 'Hostage Taking (Barricade Incident)'), (6, 'Hostage Taking (Kidnapping)'), (7, 'Facility/Infrastructure Attack'), (8, 'Unarmed Assault'), (9, 'Unknown')]


In [101]:
# AttackType 테이블 확인
attack_type_desc_query = """
    DESC AttackType
"""
cursor.execute(attack_type_desc_query)
attack_type_desc = cursor.fetchall()

for desc in attack_type_desc:
    print(desc)

('attacktype_code', b'int', 'NO', 'PRI', None, '')
('attacktype_desc', b'varchar(35)', 'NO', '', None, '')


In [103]:
# 중복을 제거한 Unique Target Type 데이터 추출
target_type_query = """
    SELECT DISTINCT targtype1, targtype1_txt FROM origin_terror_data 
    WHERE targtype1 IS NOT NULL ORDER BY targtype1
"""
cursor.execute(target_type_query)
target_type_data = cursor.fetchall()
print(target_type_data)

[(1, 'Business'), (2, 'Government (General)'), (3, 'Police'), (4, 'Military'), (5, 'Abortion Related'), (6, 'Airports & Aircraft'), (7, 'Government (Diplomatic)'), (8, 'Educational Institution'), (9, 'Food or Water Supply'), (10, 'Journalists & Media'), (11, 'Maritime'), (12, 'NGO'), (13, 'Other'), (14, 'Private Citizens & Property'), (15, 'Religious Figures/Institutions'), (16, 'Telecommunication'), (17, 'Terrorists/Non-State Militia'), (18, 'Tourists'), (19, 'Transportation'), (20, 'Unknown'), (21, 'Utilities'), (22, 'Violent Political Party')]


In [106]:
# TargetType 테이블에 데이터 입력
insert_query = """
    INSERT INTO TargetType (targtype_code, targtype_desc) VALUES (%s, %s)
"""

for data in target_type_data:
    cursor.execute(insert_query, data)
    print(data)

(1, 'Business')
(2, 'Government (General)')
(3, 'Police')
(4, 'Military')
(5, 'Abortion Related')
(6, 'Airports & Aircraft')
(7, 'Government (Diplomatic)')
(8, 'Educational Institution')
(9, 'Food or Water Supply')
(10, 'Journalists & Media')
(11, 'Maritime')
(12, 'NGO')
(13, 'Other')
(14, 'Private Citizens & Property')
(15, 'Religious Figures/Institutions')
(16, 'Telecommunication')
(17, 'Terrorists/Non-State Militia')
(18, 'Tourists')
(19, 'Transportation')
(20, 'Unknown')
(21, 'Utilities')
(22, 'Violent Political Party')


In [112]:
# TargetType 테이블 확인
target_type_desc_query = """
    DESC TargetType
"""
cursor.execute(target_type_desc_query)
target_type_desc = cursor.fetchall()

for desc in target_type_desc:
    print(desc)

('targtype_code', b'int', 'NO', 'PRI', None, '')
('targtype_desc', b'varchar(32)', 'NO', '', None, '')


In [108]:
# 중복을 제거한 Unique Weapon Type 데이터 추출
weapon_type_query = """
    SELECT DISTINCT weaptype1, weaptype1_txt FROM origin_terror_data 
    WHERE weaptype1 IS NOT NULL ORDER BY weaptype1
"""
cursor.execute(weapon_type_query)
weapon_type_data = cursor.fetchall()
print(weapon_type_data)

[(1, 'Biological'), (2, 'Chemical'), (3, 'Radiological'), (5, 'Firearms'), (6, 'Explosives'), (7, 'Fake Weapons'), (8, 'Incendiary'), (9, 'Melee'), (10, 'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)'), (11, 'Sabotage Equipment'), (12, 'Other'), (13, 'Unknown')]


In [109]:
# WeaponType 테이블에 데이터 입력
insert_query = """
    INSERT INTO WeaponType (weaptype_code, weaptype_desc) VALUES (%s, %s)
"""

for data in weapon_type_data:
    cursor.execute(insert_query, data)
    print(data)

(1, 'Biological')
(2, 'Chemical')
(3, 'Radiological')
(5, 'Firearms')
(6, 'Explosives')
(7, 'Fake Weapons')
(8, 'Incendiary')
(9, 'Melee')
(10, 'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)')
(11, 'Sabotage Equipment')
(12, 'Other')
(13, 'Unknown')


In [111]:
# WeaponType 테이블 확인
weapon_type_desc_query = """
    DESC WeaponType
"""
cursor.execute(weapon_type_desc_query)
weapon_type_desc = cursor.fetchall()

for desc in weapon_type_desc:
    print(desc)

('weaptype_code', b'int', 'NO', 'PRI', None, '')
('weaptype_desc', b'varchar(80)', 'NO', '', None, '')


문제 4. TerrorData 테이블을 만들고 앞서 만들어둔 테이블과 관계를 설정하도록 하겠습니다. 
문제 4-1. TerrorData 테이블을 앞의 그림과 같이 생성하세요. 

- origin_terror_data 를 분석하여 데이터 타입을 정의하세요. 
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요. 

참고> 
- TerrorData.terror id : Auto Increment 
- TerrorData.city_name    = origin_terror_data.city
- TerrorData.target       = orgin_terror_data.target1
- TerrorData.group_name   = origin_terror_data.gname
- TerrorData.kill_count   = origin_terror_data.nkill
- TerrorData.wound_count  = origin_terror_data.nwound
- TerrorData.motive       = origin_terror_data.motive
- TerrorData.summary      = origin_terror_data.summary
- TerrorData.latitude     = origin_terror_data_latitude
- TerrorData.longitude    = origin_terror_data_longitude
- TerrorData.terror_date  = origin_terror_data.iyear + origin_terror_Data.imonth + origin_terror_data.iday (Date Type)

In [114]:
# TerrorData 테이블 생성하기
terror_data_query = """
    CREATE TABLE TerrorData (
        terror_id INT AUTO_INCREMENT PRIMARY KEY,
        terror_date DATE NOT NULL,
        region_code INT NOT NULL,
        country_code INT NOT NULL,
        city VARCHAR(65),
        target VARCHAR(350),
        group_name VARCHAR(120),
        targtype_code INT NOT NULL,
        attacktype_code INT NOT NULL,
        weaptype_code INT NOT NULL,
        kill_count INT NOT NULL DEFAULT 0,
        wound_count INT NOT NULL DEFAULT 0,
        motive VARCHAR(900),
        summary VARCHAR(2450),
        latitude DECIMAL(16, 14),
        longitude DECIMAL(17, 14),
        CONSTRAINT FK_TerrorData_Region FOREIGN KEY (region_code) REFERENCES Region(region_code),
        CONSTRAINT FK_TerrorData_Country FOREIGN KEY (country_code) REFERENCES Country(country_code),
        CONSTRAINT FK_TerrorData_TargType FOREIGN KEY (targtype_code) REFERENCES TargetType(targtype_code),
        CONSTRAINT FK_TerrorData_AttackType FOREIGN KEY (attacktype_code) REFERENCES AttackType(attacktype_code),
        CONSTRAINT FK_TerrorData_WeapType FOREIGN KEY (weaptype_code) REFERENCES WeaponType(weaptype_code)
    )
"""
# TerrorData 테이블 생성 쿼리문 실행
cursor.execute(terror_data_query)

ProgrammingError: 1050 (42S01): Table 'TerrorData' already exists

In [116]:
# TerrorData Desc
cursor.execute("DESCRIBE TerrorData")
terror_data_schema = cursor.fetchall()
for schema in terror_data_schema:
    print(schema)

('terror_id', b'int', 'NO', 'PRI', None, 'auto_increment')
('terror_date', b'date', 'NO', '', None, '')
('region_code', b'int', 'NO', 'MUL', None, '')
('country_code', b'int', 'NO', 'MUL', None, '')
('city', b'varchar(65)', 'YES', '', None, '')
('target', b'varchar(350)', 'YES', '', None, '')
('group_name', b'varchar(120)', 'YES', '', None, '')
('targtype_code', b'int', 'NO', 'MUL', None, '')
('attacktype_code', b'int', 'NO', 'MUL', None, '')
('weaptype_code', b'int', 'NO', 'MUL', None, '')
('kill_count', b'int', 'NO', '', b'0', '')
('wound_count', b'int', 'NO', '', b'0', '')
('motive', b'varchar(900)', 'YES', '', None, '')
('summary', b'varchar(2450)', 'YES', '', None, '')
('latitude', b'decimal(16,14)', 'YES', '', None, '')
('longitude', b'decimal(17,14)', 'YES', '', None, '')


In [126]:
# max length 체크 
# city의 max length 체크
cursor.execute("SELECT MAX(LENGTH(city)) FROM origin_terror_data")
max_city_len = cursor.fetchone()[0]

# target1의 max length 체크
cursor.execute("SELECT MAX(LENGTH(target1)) FROM origin_terror_data")
max_target1_len = cursor.fetchone()[0]

# gname의 max length 체크
cursor.execute("SELECT MAX(LENGTH(gname)) FROM origin_terror_data")
max_gname_len = cursor.fetchone()[0]

# summary의 max length 체크
cursor.execute("SELECT MAX(LENGTH(summary)) FROM origin_terror_data")
max_summary_len = cursor.fetchone()[0]

# motive의 max length 체크
cursor.execute("SELECT MAX(LENGTH(motive)) FROM origin_terror_data")
max_motive_len = cursor.fetchone()[0]

print("city:", max_city_len)
print("target1:", max_target1_len)
print("gname:", max_gname_len)
print("summary:", max_summary_len)
print("motive:", max_motive_len)


city: 65
target1: 343
gname: 112
summary: 2431
motive: 899


문제 4-2. origin_terror_data 테이블에서 Terror Data를 추출하여 문제 4-1.에서 생성한 테이블에 입력하고 확인하세요.

- 앞서 생성한 Region, Country, AttackType, TargetType, WeaponType 데이터와의 관계에 주의하세요.
- Count 값을 가지는 칼럼의 값이 null 인 경우, 0으로 예외처리 해주세요.
- 위도 경도 데이터 중 범위를 넘어서는 데이터가 존재합니다. 이 경우, null 값으로 예외처리 해주세요. (위도 경도 범위 : 구글링해보세요.)
- terror_date 칼럼의 경우, origin_terror_data 의 연, 월, 일 정보를 조합하여 date type 으로 정의해주세요. (Format : ‘YYYY-mm-dd’)
- origin_terror_data 의 월, 일 정보중 값이 0 인 경우 date type 으로 변환되지 않습니다. 이 경우, 1 로 예외처리 해주세요.
- 데이터 입력까지 완료한 이후, origin_terror_data 테이블을 삭제하고 확인하세요.

제출 7. Table 생성 및 조회 결과 : DESC TerrorData, SELECT COUNT(*) FROM TerrorData 

제출 8. 관련 코드 및 실행 결과 (ipynb)

In [210]:
query = """
INSERT INTO TerrorData (region_code, country_code, attacktype_code, targtype_code, weaptype_code, terror_date, city, target, group_name, kill_count, wound_count, motive, summary, latitude, longitude)
SELECT c.region_code, td.country, at.attacktype_code, tt.targtype_code, wt.weaptype_code,
    CONCAT(CASE WHEN td.iyear = 0 THEN '0001' ELSE LPAD(td.iyear, 4, '0') END, '-', CASE WHEN td.imonth = 0 THEN '01' ELSE LPAD(td.imonth, 2, '0') END, '-', CASE WHEN td.iday = 0 THEN '01' ELSE LPAD(td.iday, 2, '0') END) AS terror_date,
    IFNULL(td.city, '') AS city,
    IFNULL(td.target1, '') AS target,
    IFNULL(td.gname, '') AS group_name,
    COALESCE(td.nkill, 0) AS kill_count,
    COALESCE(td.nwound, 0) AS wound_count,
    IFNULL(td.motive, '') AS motive,
    IFNULL(td.summary, '') AS summary,
    IF(td.latitude > 90 OR td.latitude < -90, NULL, td.latitude) AS latitude,
    IF(td.longitude > 180 OR td.longitude < -180, NULL, td.longitude) AS longitude
FROM origin_terror_data td
INNER JOIN Country c ON td.country_txt = c.country_name
LEFT JOIN AttackType at ON td.attacktype1_txt = at.attacktype_desc
LEFT JOIN TargetType tt ON td.targtype1_txt = tt.targtype_desc
LEFT JOIN WeaponType wt ON td.weaptype1_txt = wt.weaptype_desc;
"""

cursor.execute(query)

In [212]:
conn.commit()

In [213]:
# TerrorData 테이블 구조 확인
cursor.execute("DESC TerrorData")
print(cursor.fetchall())

[('terror_id', b'int', 'NO', 'PRI', None, 'auto_increment'), ('terror_date', b'date', 'NO', '', None, ''), ('region_code', b'int', 'NO', 'MUL', None, ''), ('country_code', b'int', 'NO', 'MUL', None, ''), ('city', b'varchar(65)', 'YES', '', None, ''), ('target', b'varchar(350)', 'YES', '', None, ''), ('group_name', b'varchar(120)', 'YES', '', None, ''), ('targtype_code', b'int', 'NO', 'MUL', None, ''), ('attacktype_code', b'int', 'NO', 'MUL', None, ''), ('weaptype_code', b'int', 'NO', 'MUL', None, ''), ('kill_count', b'int', 'NO', '', b'0', ''), ('wound_count', b'int', 'NO', '', b'0', ''), ('motive', b'varchar(900)', 'YES', '', None, ''), ('summary', b'varchar(2450)', 'YES', '', None, ''), ('latitude', b'decimal(16,14)', 'YES', '', None, ''), ('longitude', b'decimal(17,14)', 'YES', '', None, '')]


In [214]:
# TerrorData 테이블 레코드 수 확인
cursor.execute("SELECT COUNT(*) FROM TerrorData")
print(cursor.fetchone())

(181691,)


In [215]:
# origin_terror_data 테이블 삭제
cursor.execute("DROP TABLE origin_terror_data")

문제 5. TerrorData 의 전체 기간에서 테러의 숫자를 연도별로 집계하여 연도별 테러 발생 건수를 조회하세요. 

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 

제출 9. 관련 코드 및 실행 결과 (ipynb)

('1970', 651)
('1971', 471)
('1972', 568)
('1973', 473)
('1974', 581)
('1975', 740)
('1976', 923)
('1977', 1319)
('1978', 1526)
('1979', 2662)
('1980', 2662)
('1981', 2586)
('1986', 2860)
('1982', 2544)
('1983', 2870)
('1984', 3495)
('1985', 2915)
('1987', 3183)
('1988', 3721)
('1989', 4324)
('1990', 3887)
('1991', 4683)
('1992', 5071)
('1994', 3456)
('1995', 3081)
('1996', 3058)
('1997', 3197)
('1998', 934)
('1999', 1395)
('2000', 1814)
('2001', 1906)
('2002', 1333)
('2003', 1278)
('2004', 1166)
('2005', 2017)
('2006', 2758)
('2007', 3242)
('2008', 4805)
('2009', 4721)
('2010', 4826)
('2011', 5076)
('2012', 8522)
('2013', 12036)
('2014', 16903)
('2015', 14965)
('2016', 13587)
('2017', 10900)


문제 6. TerrorData 에서 테러가 가장 많이 일어난 순서로 국가를 정렬하여 상위 10위 국가를 조회하세요. 

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 
- 국가 정보는 이름으로 출력하세요.

제출 10. 관련 코드 및 실행 결과 (ipynb)

In [222]:
# 테러가 많이 일어난 상위 10위 Region

query = '''
    SELECT c.region_name, COUNT(*) as cnt
    FROM TerrorData td
    JOIN Region c ON td.region_code = c.region_code
    GROUP BY c.region_name
    ORDER BY cnt DESC
'''

cursor.execute(query)
result = cursor.fetchall()

for row in result:
    print(row)

('Middle East & North Africa', 50474)
('South Asia', 44974)
('South America', 18978)
('Sub-Saharan Africa', 17550)
('Western Europe', 16639)
('Southeast Asia', 12485)
('Central America & Caribbean', 10344)
('Eastern Europe', 5144)
('North America', 3456)
('East Asia', 802)
('Central Asia', 563)
('Australasia & Oceania', 282)


In [220]:
# 테러가 많이 일어난 상위 10위 Country 

query = '''
    SELECT c.country_name, COUNT(*) as cnt
    FROM TerrorData td
    JOIN Country c ON td.country_code = c.country_code
    GROUP BY c.country_name
    ORDER BY cnt DESC
    LIMIT 10
'''

cursor.execute(query)
result = cursor.fetchall()

for row in result:
    print(row)

('Iraq', 24636)
('Pakistan', 14368)
('Afghanistan', 12731)
('India', 11960)
('Colombia', 8306)
('Philippines', 6908)
('Peru', 6096)
('El Salvador', 5320)
('United Kingdom', 5235)
('Turkey', 4292)


문제 7. TerrorData 에서 테러가 가장 많이 일어난 상위 10위 국가에 대해 국가별로 사망자수와 부상자수, 사상자수(사망자수 + 부상자수)를 조회하세요. 

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 
- 국가 정보는 이름으로 출력하세요.

제출 11. 관련 코드 및 실행 결과 (ipynb)

In [233]:
query = '''
SELECT c.country_name AS country, COUNT(*) as cnt, SUM(td.kill_count) AS deaths, SUM(td.wound_count) AS injuries, SUM(td.kill_count + td.wound_count) AS casualties
FROM TerrorData td
INNER JOIN Country c ON td.country_code = c.country_code
GROUP BY c.country_name
ORDER BY cnt DESC
LIMIT 10;

'''

cursor.execute(query)
result = cursor.fetchall()

for row in result:
    print(row)

('Iraq', 24636, Decimal('78589'), Decimal('134690'), Decimal('213279'))
('Pakistan', 14368, Decimal('23822'), Decimal('42038'), Decimal('65860'))
('Afghanistan', 12731, Decimal('39384'), Decimal('44277'), Decimal('83661'))
('India', 11960, Decimal('19341'), Decimal('28980'), Decimal('48321'))
('Colombia', 8306, Decimal('14698'), Decimal('10328'), Decimal('25026'))
('Philippines', 6908, Decimal('9559'), Decimal('13367'), Decimal('22926'))
('Peru', 6096, Decimal('12771'), Decimal('4078'), Decimal('16849'))
('El Salvador', 5320, Decimal('12053'), Decimal('5062'), Decimal('17115'))
('United Kingdom', 5235, Decimal('3410'), Decimal('6106'), Decimal('9516'))
('Turkey', 4292, Decimal('6888'), Decimal('9899'), Decimal('16787'))


문제 8. 지역별 테러 공격 형태에 따른 사망자수, 부상자수, 사상자수를 조회하세요.

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 
- 지역 정보(Region)는 이름으로 출력하세요.

제출 12. 관련 코드 및 실행 결과 (ipynb)

In [254]:
query = '''
SELECT 
    r.region_name, 
    at.attacktype_desc, 
    SUM(td.kill_count) AS total_kills, 
    SUM(td.wound_count) AS total_wounds,
    SUM(td.kill_count + td.wound_count) AS total_casualties
FROM Region r
JOIN Country c ON r.region_code = c.region_code
JOIN TerrorData td ON c.country_code = td.country_code
JOIN AttackType at ON td.attacktype_code = at.attacktype_code
GROUP BY r.region_name, at.attacktype_desc
ORDER BY r.region_name, at.attacktype_desc
'''

cursor.execute(query)
result = cursor.fetchall()

for row in result:
    print(row)

('Australasia & Oceania', 'Armed Assault', Decimal('78'), Decimal('87'), Decimal('165'))
('Australasia & Oceania', 'Assassination', Decimal('34'), Decimal('9'), Decimal('43'))
('Australasia & Oceania', 'Bombing/Explosion', Decimal('9'), Decimal('77'), Decimal('86'))
('Australasia & Oceania', 'Facility/Infrastructure Attack', Decimal('0'), Decimal('13'), Decimal('13'))
('Australasia & Oceania', 'Hijacking', Decimal('0'), Decimal('0'), Decimal('0'))
('Australasia & Oceania', 'Hostage Taking (Barricade Incident)', Decimal('6'), Decimal('7'), Decimal('13'))
('Australasia & Oceania', 'Hostage Taking (Kidnapping)', Decimal('7'), Decimal('20'), Decimal('27'))
('Australasia & Oceania', 'Unarmed Assault', Decimal('1'), Decimal('41'), Decimal('42'))
('Australasia & Oceania', 'Unknown', Decimal('15'), Decimal('6'), Decimal('21'))
('Central America & Caribbean', 'Armed Assault', Decimal('23233'), Decimal('5459'), Decimal('28692'))
('Central America & Caribbean', 'Assassination', Decimal('1699'), D

---

# reference

민형동 (2020). Global Terrorism Data를 활용한 국가별 테러유형 분석. 한국공안행정학회, 82, 63-84. 

한인택 (2017). Global Terrorism Database를 활용한 테러리즘의 현황분석. 

민형기 (2019). 전세계 테러 데이터 분석 (따라하기). https://pinkwink.kr/1214 

ASHWINI SWSAIN (2018). Terrorism Around The World. https://www.kaggle.com/code/ash316/terrorism-around-the-world/notebook

START (National Consortium for the Study of Terrorism and Responses to Terrorism). (2021). *Global Terrorism Database* (GTD) [Data set]. University of Maryland. https://www.start.umd.edu/gtd

START (National Consortium for the Study of Terrorism and Responses to Terrorism). (2021, August). *Global Terrorism Database codebook: Methodology, inclusion criteria, and variables.* University of Maryland. https://www.start.umd.edu/gtd/downloads/Codebook.pdf 
